Task order: Task2, Task5, Task3
The algorithms in the 3 sections use the same variable names. Please run the whole program at once or do preprocessing + frequent itemset generation + frequent rule generation together.

In [547]:
import numpy as np
import pandas as pd
import itertools

supermarket = pd.read_csv("supermarket.csv", sep='\n', header=None,)
all_transaction_dict = {}
dict_by_words = {}
for i in range(len(supermarket)):
    row = supermarket.iloc[i].str.split(",")[0]
    all_transaction_dict[i] = np.array(row)
    for word in row:
        if word in dict_by_words:
            dict_by_words[word].append(i)
        else:
            dict_by_words[word] = []
            dict_by_words[word].append(i)


 The code directly below is the Apriori algorithm a detailed description is provided in the report

In [531]:
minsup = 0.15
minconf = 0.80
minlift = 0
sup_value = minsup * len(supermarket)
print("minsup in occurences: ",sup_value)
#Apriori implementation,
candidates = np.array([])
common_sets = {}
k = 2
unique = set(dict_by_words.keys())
for value in unique:
    if len(dict_by_words[value]) > sup_value:
        candidates = np.append(candidates,value)
original = candidates
individuals = np.array([str(a) for a in range(len(original))])
for code in individuals:
    common_sets[code] = dict_by_words[original[int(code)]]
pruned = individuals
print("Preprocessing complete")
results = np.array(individuals)
forbidden = {}
while len(pruned) > 0:
    covered = np.array([])
    print("Iteration begins")
    print(len(pruned))
    candidates = np.array([])
    for z in range(len(pruned)):
        element = pruned[z]
        set1 = set(common_sets[element])
        for q in range(len(individuals)):
            possible = individuals[q]
            if k > 2 and possible in forbidden["".join(element)]:
                continue
            set2 = set(common_sets[possible])
            set3 = set1 & set2
            if len(set3) > sup_value and possible not in element:
                temp = element.split(",") + [possible]
                covered = np.append(covered,temp)
                full = sorted(temp)
                full = ",".join(full)
                if full not in common_sets:
                    candidates = np.append(candidates, full)
                    common_sets[full] = set3 
                    results = np.append(results,full)
            else:
                if k == 2:
                    if element not in forbidden:
                        forbidden[element] = np.array([])
                    forbidden[element] = np.append(forbidden[element],possible)
    for r in candidates:
        no = np.array([])
        q_list = r.split(",")
        for u in range(len(q_list)):
            no = np.append(no,forbidden[q_list[u]])
        forbidden["".join(r)] = no
    covered = set(covered)
    individuals = np.array(list(covered))
    pruned = candidates
    k += 1 
print("DONE")
print(len(results))
#Generating unique itemsets and possible association rules
print(len(results), "frequent itemsets")
total = []
for value in results:
    qq = value.split(",")
    for i in range(1,len(qq)):
        xs = itertools.combinations(qq, i)
        for v in xs:
            copy1 = value.split(",")
            for x in v:
                copy1.pop(copy1.index(x))
            if (list(v),copy1) not in total:
                total.append((list(v),copy1))
print(len(total), "possible association rules")

minsup in occurences:  694.05
Preprocessing complete
Iteration begins
44
Iteration begins
379
Iteration begins
909
Iteration begins
629
Iteration begins
104
Iteration begins
1
DONE
2066
2066 frequent itemsets
18200 possible association rules


In [532]:
#Generating frequent rules
final_rules  = []
for pair in total:
    first = pair[0]
    second = pair[1]
    code1 = ",".join(sorted(first))
    code2 = ",".join(sorted(second))
    text1 = ", ".join([original[int(x)] for x in first])
    text2 = ", ".join([original[int(x)] for x in second])
    full = ",".join(sorted(first + second))
    p_a = len(common_sets[code1])
    p_b = len(common_sets[code2])
    p_ab = len(common_sets[full])
    confidence = p_ab / p_a
    lift = confidence / (p_b/len(supermarket))
    if confidence >= minconf and lift >= minlift:
        full_text = "(" + text1 + " -> " + text2 + ")"
        final_rules.append([full_text,lift,confidence])
final_rules = sorted(final_rules, key=lambda e: (len(final_rules)-len(e[0]), e[1], e[2]))
print_list = [e[0] for e in final_rules]
print(print_list)

['(juice-sat-cord-ms, party snack foods, biscuits, frozen foods -> bread and cake)', '(bread and cake, juice-sat-cord-ms, party snack foods, frozen foods -> biscuits)', '(juice-sat-cord-ms, baking needs, vegetables, frozen foods -> bread and cake)', '(sauces-gravy-pkle, baking needs, vegetables, frozen foods -> bread and cake)', '(party snack foods, baking needs, vegetables, frozen foods -> bread and cake)', '(baking needs, tissues-paper prd, vegetables, frozen foods -> bread and cake)', '(milk-cream, baking needs, tissues-paper prd, frozen foods -> bread and cake)', '(sauces-gravy-pkle, juice-sat-cord-ms, party snack foods -> bread and cake)', '(juice-sat-cord-ms, party snack foods, tissues-paper prd -> bread and cake)', '(sauces-gravy-pkle, party snack foods, tissues-paper prd -> bread and cake)', '(sauces-gravy-pkle, biscuits, baking needs, frozen foods -> bread and cake)', '(juice-sat-cord-ms, biscuits, baking needs, frozen foods -> bread and cake)', '(milk-cream, juice-sat-cord-ms

In [533]:
print("Total Frequent Rules: ",len(final_rules))

Total Frequent Rules:  955


# FROM HERE ONWARDS IS TASK 5

In [577]:
#make sure to run the supermarket.csv preprocessing section, not the Task 1 dataset
minsup = 0.15
minconf = 0.80
minlift = 0
minrelativesup = 0.5
sup_value = minsup * len(supermarket)
minrelativesupvalue = minrelativesup * len(supermarket)
print(minrelativesupvalue)

#task 5 min_relative_sup
candidates = np.array([])
common_sets = {}
k = 2
unique = set(dict_by_words.keys())
for value in unique:
    if len(dict_by_words[value]) > sup_value:
        candidates = np.append(candidates,value)
original = candidates
individuals = np.array([str(a) for a in range(len(original))])
for code in individuals:
    common_sets[code] = dict_by_words[original[int(code)]]
pruned = individuals
print("Preprocessing complete")
results = np.array(individuals)
forbidden = {}
while len(pruned) > 0:
    covered = np.array([])
    print("Iteration begins")
    print(len(pruned))
    candidates = np.array([])
    for z in range(len(pruned)):
        element = pruned[z]
        set1 = set(common_sets[element])
        for q in range(len(individuals)):
            possible = individuals[q]
            if k > 2 and possible in forbidden["".join(element)]:
                continue
            set2 = set(common_sets[possible])
            set3 = set1 & set2
            if len(set3) > sup_value and possible not in element:
                truth = True
                temp = element.split(",") + [possible]
                covered = np.append(covered,temp)
                full = sorted(temp)
                full = ",".join(full)
                if k > 2:                                                         #########
                    k_minus = itertools.combinations(full.split(","),k-1)         #########
                    largest = 0                                                   #########
                    for m in k_minus:                                             #########
                        term = "".join(m)                                         #########
                        if term in common_sets.keys():                            #########
                            temp_val = len(common_sets[term])                     #########
                            if temp_val > largest:                                #########
                                largest = temp_val                                #########
                    if largest < minrelativesupvalue:                             #########
                        truth = False                                             #########
                if full not in common_sets and truth:
                    candidates = np.append(candidates, full)
                    common_sets[full] = set3 
                    results = np.append(results,full)
            else:
                if k == 2:
                    if element not in forbidden:
                        forbidden[element] = np.array([])
                    forbidden[element] = np.append(forbidden[element],possible)
    for r in candidates:
        no = np.array([])
        q_list = r.split(",")
        for u in range(len(q_list)):
            no = np.append(no,forbidden[q_list[u]])
        forbidden["".join(r)] = no
    covered = set(covered)
    individuals = np.array(list(covered))
    pruned = candidates
    k += 1 
print("DONE")
#Generating unique itemsets and possible association rules
print(len(results), "frequent itemsets")
total = []
for value in results:
    qq = value.split(",")
    for i in range(1,len(qq)):
        xs = itertools.combinations(qq, i)
        for v in xs:
            copy1 = value.split(",")
            for x in v:
                copy1.pop(copy1.index(x))
            if (list(v),copy1) not in total:
                total.append((list(v),copy1))
print(len(total), "possible association rules")

2313.5
Preprocessing complete
Iteration begins
44
Iteration begins
379
DONE
423 frequent itemsets
758 possible association rules


In [578]:
#Generating frequent rules
final_rules  = []
for pair in total:
    first = pair[0]
    second = pair[1]
    code1 = ",".join(sorted(first))
    code2 = ",".join(sorted(second))
    text1 = ", ".join([original[int(x)] for x in first])
    text2 = ", ".join([original[int(x)] for x in second])
    full = ",".join(sorted(first + second))
    p_a = len(common_sets[code1])
    p_b = len(common_sets[code2])
    p_ab = len(common_sets[full])
    confidence = p_ab / p_a
    lift = confidence / (p_b/len(supermarket))
    if confidence >= minconf and lift >= minlift:
        full_text = "(" + text1 + " -> " + text2 + ")"
        final_rules.append([full_text,lift,confidence])
final_rules = sorted(final_rules, key=lambda e: (len(final_rules)-len(e[0]), e[1], e[2]))
print_list = [e[0] for e in final_rules]
print(print_list)

['(canned fish-meat -> bread and cake)', '(jams-spreads -> bread and cake)', '(canned fruit -> bread and cake)', '(total = high -> bread and cake)', '(small goods -> bread and cake)', '(margarine -> bread and cake)']


In [579]:
print("Total Frequent Rules: ",len(final_rules))

Total Frequent Rules:  6


# FROM HERE ONWARDS IS TASK 3
I have copy pasted everything from my Task 2 Apriori implementation. The only difference is the preprocessing.

In [537]:
minsup = 0.15
minconf = 0.80
sup_value = minsup * len(lines_list)


line1 = ["A", "B", "C", "D", "F"]
line2 = ["A", "B", "C", "D"]
line3 = ["A", "B", "C", "D"]
line4 = ["A", "B"]
line5 = ["B", "C", "E" ]
lines = line1 + line2 + line3 + line4 + line5
lines = list(set(lines))
lines_list = [line1,line2,line3,line4,line5]

dict_by_words = {}
for i in range(len(lines_list)):
    row = lines_list[i]
    for word in row:
        if word in dict_by_words:
            dict_by_words[word].append(i)
        else:
            dict_by_words[word] = []
            dict_by_words[word].append(i)
print(dict_by_words)
print("minsup in occurences: ",sup_value)

#Apriori implementation, a detailed description is provided in the report
candidates = np.array([])
common_sets = {}
k = 2
unique = set(dict_by_words.keys())
for value in unique:
    if len(dict_by_words[value]) > sup_value:
        candidates = np.append(candidates,value)
original = candidates
individuals = np.array([str(a) for a in range(len(original))])
for code in individuals:
    common_sets[code] = dict_by_words[original[int(code)]]
pruned = individuals
print("Preprocessing complete")
results = np.array(individuals)
forbidden = {}
while len(pruned) > 0:
    covered = np.array([])
    print("Iteration begins")
    print(len(pruned))
    candidates = np.array([])
    for z in range(len(pruned)):
        element = pruned[z]
        set1 = set(common_sets[element])
        for q in range(len(individuals)):
            possible = individuals[q]
            if k > 2 and possible in forbidden["".join(element)]:
                continue
            set2 = set(common_sets[possible])
            set3 = set1 & set2
            if len(set3) >= sup_value and possible not in element:
                temp = element.split(",") + [possible]
                covered = np.append(covered,temp)
                full = sorted(temp)
                full = ",".join(full)
                if full not in common_sets:
                    candidates = np.append(candidates, full)
                    common_sets[full] = set3 
                    results = np.append(results,full)
            else:
                if k == 2:
                    if element not in forbidden:
                        forbidden[element] = np.array([])
                    forbidden[element] = np.append(forbidden[element],possible)
    for r in candidates:
        no = np.array([])
        q_list = r.split(",")
        for u in range(len(q_list)):
            no = np.append(no,forbidden[q_list[u]])
        forbidden["".join(r)] = no
    covered = set(covered)
    individuals = np.array(list(covered))
    pruned = candidates
    k += 1 
print("DONE")
print(len(results))
#Generating unique itemsets and possible association rules
print(len(results), "frequent itemsets")
total = []
for value in results:
    qq = value.split(",")
    for i in range(1,len(qq)):
        xs = itertools.combinations(qq, i)
        for v in xs:
            copy1 = value.split(",")
            for x in v:
                copy1.pop(copy1.index(x))
            if (list(v),copy1) not in total:
                total.append((list(v),copy1))
print(len(total), "possible association rules")

{'A': [0, 1, 2, 3], 'B': [0, 1, 2, 3, 4], 'C': [0, 1, 2, 4], 'D': [0, 1, 2], 'F': [0], 'E': [4]}
minsup in occurences:  0.75
Preprocessing complete
Iteration begins
6
Iteration begins
12
Iteration begins
11
Iteration begins
5
Iteration begins
1
DONE
35
35 frequent itemsets
190 possible association rules


In [538]:
#Generating frequent rules
final_rules  = []
for pair in total:
    first = pair[0]
    second = pair[1]
    code1 = ",".join(sorted(first))
    code2 = ",".join(sorted(second))
    text1 = ", ".join([original[int(x)] for x in first])
    text2 = ", ".join([original[int(x)] for x in second])
    full = ",".join(sorted(first + second))
    p_a = len(common_sets[code1])
    p_b = len(common_sets[code2])
    p_ab = len(common_sets[full])
    confidence = p_ab / p_a
    lift = confidence / (p_b/len(supermarket))
    if confidence >= minconf and lift >= minlift:
        full_text = "(" + text1 + " -> " + text2 + ")"
        final_rules.append([full_text,lift,confidence])
final_rules = sorted(final_rules, key=lambda e: (len(final_rules)-len(e[0]), e[1], e[2]))
print_list = [e[0] for e in final_rules]
print(print_list)

['(A, D, C, F -> B)', '(A, D, F -> B, C)', '(D, C, F -> B, A)', '(B, A, D, F -> C)', '(B, D, C, F -> A)', '(F -> B, A, D, C)', '(B, F -> A, D, C)', '(A, F -> B, D, C)', '(D, F -> B, A, C)', '(C, F -> B, A, D)', '(B, A, F -> D, C)', '(B, D, F -> A, C)', '(B, C, F -> A, D)', '(A, C, F -> B, D)', '(B, A, C, F -> D)', '(A, C, F -> B)', '(A, D, F -> B)', '(A, D, C -> B)', '(D, C, F -> B)', '(A, F -> B, C)', '(C, F -> B, A)', '(B, A, F -> C)', '(B, C, F -> A)', '(D, F -> B, A)', '(B, D, F -> A)', '(A, D -> B, C)', '(D, C -> B, A)', '(B, A, D -> C)', '(B, D, C -> A)', '(D, F -> B, C)', '(B, D, F -> C)', '(A, D, F -> C)', '(D, C, F -> A)', '(F -> B, A, C)', '(B, F -> A, C)', '(F -> B, A, D)', '(B, F -> A, D)', '(A, F -> B, D)', '(B, A, F -> D)', '(D -> B, A, C)', '(B, D -> A, C)', '(A, C -> B, D)', '(B, A, C -> D)', '(F -> B, D, C)', '(B, F -> D, C)', '(C, F -> B, D)', '(B, C, F -> D)', '(F -> A, D, C)', '(A, F -> D, C)', '(D, F -> A, C)', '(C, F -> A, D)', '(A, C, F -> D)', '(A, F -> B)', '(A

In [539]:
print("Total Frequent Rules: ",len(print_list))

Total Frequent Rules:  97
